In [ ]:
from keras import layers
from keras import models
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def load_cifar10(subset = True):
    """
    Loads a training, validation, and test set of CIFAR10 images.
    
    When subset=TRUE:
    Returns only a subset of the mnist dataset.
    Especially important to use if you are on datahub and only have 1-2GB of memory.
    """
    if subset:
        N_TRAIN = 8000
        N_VALIDATION = 2000
        N_TEST = 2000
    else:
        N_TRAIN = 40000
        N_VALIDATION = 10000
        N_TEST = 10000
    
    (x_train_and_val, y_train_and_val), (x_test, y_test) = cifar10.load_data()
    
    x_train = x_train_and_val[:N_TRAIN,:,:]
    y_train = y_train_and_val[:N_TRAIN]
    
    x_val = x_train_and_val[N_TRAIN: N_TRAIN + N_VALIDATION,:,:]
    y_val = y_train_and_val[N_TRAIN: N_TRAIN + N_VALIDATION]
    
    x_test = x_test[:N_TEST]
    y_test = y_test[:N_TEST]
    
    return x_train, y_train, x_val, y_val, x_test, y_test

x_train, y_train, x_val, y_val, x_test, y_test = load_cifar10()

def three_dim_transform(x_data, y_data):
    """
    Transforms image data:
        - Scales pixel values between [0,1]
    Transforms target data (ydata):
        - Formats targets as one hot encoded columns
    """
    
    x = {}
    for name, partition in zip(["x_train", "x_val", "x_test"],x_data):
        scaled = partition.astype('float32') / 255
        x[name] = scaled
    
    y = {}
    for name, partition in zip(["y_train", "y_val", "y_test"],y_data):
        y[name] = to_categorical(partition)
    
    return x['x_train'], y['y_train'], x['x_val'], y['y_val'], x['x_test'], y['y_test']

x_train_trans, y_train_trans, x_val_trans, y_val_trans, x_test_trans, y_test_trans = three_dim_transform([x_train, x_val, x_test],
                                                                                                    [y_train, y_val, y_test])

## Challenge 1: Translate Classes

Create a function `translate_class()` that uses the correct class name for the target classes (truck, horse, etc..).
- Use the [keras CIFAR10 documentation](https://keras.io/api/datasets/cifar10/) as a guide to know how the classes are labeled.
- The function should take a class index [0-9]
- The function should return the correct corresponding CIFAR10 class category.

In [ ]:
def translate_class(y):
    """
    Takes a class index [0-9] and returns the CIFAR10 class category.
    """
    # Create a list of categories
    categories = ["airplane", 
                 "automobile",
                 "bird",
                 "cat",
                 "deer",
                 "dog",
                 "frog",
                 "horse",
                 "ship",
                 "truck"]
    
    return categories[y]
    
    
def translate_classes_fancy(y):
    """
    Use a key-value paired dictionary to translate target class
    """
    # Create a list of categories
    categories = ["airplane", 
                 "automobile",
                 "bird",
                 "cat",
                 "deer",
                 "dog",
                 "frog",
                 "horse",
                 "ship",
                 "truck"]
    
    # Use a dictionary comprehesion to attach class number to category
    category_dict = {key : value for key, value in zip(list(range(10)), categories)}
    
    return category_dict[y]

## Challenge 2: Plotting Image Classes

Create a new function `my_imageplotter()` that reuses code from our `plot_images()` function and incorporates the `translate_class()` function to give us the correct class titles in our images.

In [ ]:
def my_imageplotter(x, y, random=False):
    """
    Plots 25 images from x data with titles set as y.
    Set random=True if you want random images rather than the first 25.
    """
    
    if random:
        indices = np.random.choice(range(y.shape[0]), 25, replace=False)
    
    else:
        indices = np.array(range(25))
    
    fig, axes = plt.subplots(5,5, figsize = (15,15))
    axes = axes.ravel()
    
    for ax, index in zip(axes, indices):
        # NEW LINE HERE
        title = translate_class(y[index][0])
        ax.imshow(x[index])
        ax.set_title(f"Class: {title}", size=15)
    
    plt.tight_layout()
    
    return plt.show()

## Challenge 3: Create a Convolutional Neural Network

1. Initialize a Convolutional Neural Network called `my_convnet` with the following architecture:
    * Conv2D layer with 32 3x3 filters and relu activation function
    * Maxpooling layer 2x2
    * Conv2D layer with 64 3x3 filters and relu activation function
    * Maxpooling layer 2x2
    * Conv2D layer with 128 3x3 filters and relu activation function
    * Maxpooling layer wtih a pool size of 2x2
    * A flattening layer
    * A dense layer with 512 neurons  and relu activation function
    * An output layer with the number of classes and softmax activation function

2. Compile with the model with:
    * rmsprop optimizer
    * categorical crossentropy loss
    * accuracy metric

3. Train the network for 20 epochs.

4. Plot the training and validation accuracy through each epoch


In [ ]:
# 3.1 Initialize a Convolutional Neural Network called `my_convnet`
my_convnet = models.Sequential()
my_convnet.add(layers.Conv2D(32, (3,3), activation= "relu", input_shape = (32, 32, 3)))
my_convnet.add(layers.MaxPooling2D(pool_size=(2, 2)))
my_convnet.add(layers.Conv2D(64, (3,3), activation= "relu"))
my_convnet.add(layers.MaxPooling2D(pool_size=(2, 2)))
my_convnet.add(layers.Conv2D(128, (3,3), activation= "relu"))
my_convnet.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten for dense layers
my_convnet.add(layers.Flatten())
my_convnet.add(layers.Dense(512, activation= "relu"))
my_convnet.add(layers.Dense(10, activation= "softmax"))


In [ ]:
# 3.2 Compile model
my_convnet.compile(optimizer= "rmsprop",
                   loss= "categorical_crossentropy",
                   metrics= ["accuracy"])

In [ ]:
# 3.3 Train the network for 20 epochs
my_convnet_history = my_convnet.fit(x_train_trans,
                             y_train_trans,
                             epochs=20,
                             batch_size = 64,
                             validation_data=(x_val_trans, y_val_trans))

In [ ]:
def plot_epoch_accuracy(history_dict):
    """
    Plots the training and validation accuracy of a neural network.
    """
    
    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, color = 'navy', alpha = 0.8, label='Training Accuracy')
    plt.plot(epochs, val_acc, color = 'green', label='Validation Accuracy')
    plt.title('Training and validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    return plt.show()

In [ ]:
# 3.4 Plot accuracy through epochs
plot_epoch_accuracy(my_convnet_history.history)